In [70]:
import xml.etree.ElementTree as ET
import os
import shutil

In [18]:
def read_xml(manga_name):
    base_path = r"D:\Project\Manga109\Manga109_released_2023_12_07\annotations.v2020.12.18"
    file_path = f"{base_path}\\{manga_name}.xml"

    with open(file_path, encoding='utf-8') as f:
        xml_data = f.read()

    return xml_data

In [ ]:
def get_folder(folder_type):
    base_path = r"D:\Project\Manga109\Manga109_released_2023_12_07"
    folder_path = f"{base_path}\\{folder_type}"

    return folder_path

In [68]:
def create_folder(manga_name, folder_type):
    base_path = r"D:\Project\Manga109\Manga109_released_2023_12_07"
    folder_path = f"{base_path}\\{folder_type}\\{manga_name}"
    
    try:
        os.makedirs(folder_path)
    except Exception as e:
        print(e)

    return folder_path

In [57]:
def save_annotation(manga_name, page, annotation):
    folder_path = create_folder(manga_name, 'labels')
    
    try:
        os.makedirs(folder_path, exist_ok=True)
    except Exception as e:
        print(e)

    file_path = f"{folder_path}\\{page}.txt"
    with open(file_path, "w") as f:
        f.write("\n".join(annotation))

In [79]:
def format_page(page):
    if len(page) == 1:
        return f'00{str(page)}'
    elif len(page) == 2:
        return f'0{str(page)}'
    else:
        return str(page)

In [77]:
BASE_PATH = r"D:\Project\Manga109\Manga109_released_2023_12_07"

def remove_no_labels(manga_name, no_label: list[int]):
    folder_path = f"{BASE_PATH}\\images\\{manga_name}"
    folder_path_dest = create_folder('YasasiiAkuma', 'images_no_labels')
    
    for page in no_label:
        shutil.move(f"{folder_path}\\{format_page(page)}.jpg", f"{folder_path_dest}\\{format_page(page)}.jpg")

In [28]:
xml_data = read_xml(manga_name='YasasiiAkuma')

In [29]:
xml_data

'<book title="YasasiiAkuma">\n  <characters>\n    <character id="0007b8ef" name="ミチスケ"/>\n    <character id="0007b8f1" name="聖先生"/>\n    <character id="0007b8f4" name="Other"/>\n    <character id="0007b903" name="神田昇治"/>\n    <character id="0007b911" name="宝"/>\n    <character id="0007b915" name="碧"/>\n    <character id="0007b987" name="いじめっ子Ｂ"/>\n    <character id="0007b98f" name="いじめっ子Ａ"/>\n    <character id="0007ba64" name="ミチ子"/>\n    <character id="0007bb54" name="神田昇治の母"/>\n    <character id="0007bdd2" name="おじいさん"/>\n    <character id="0007be1b" name="はやみ義明"/>\n    <character id="0007be36" name="はやみ義明の妻"/>\n    <character id="0007bf18" name="はやみ義明の母"/>\n    <character id="0007bf88" name="ブァッサーゴ"/>\n    <character id="0007c042" name="はやみ義明の娘"/>\n    <character id="0007c065" name="はやみ義明の息子"/>\n    <character id="0007c1d3" name="橋之助"/>\n    <character id="0007c2c9" name="医者A"/>\n    <character id="0007c2e0" name="医者B"/>\n    <character id="0007c5e1" name="千盛"/>\n    <character id="

In [44]:
root = ET.fromstring(xml_data)

pages = root.find('pages')

In [59]:
def annotation(data, img_width, img_height, class_id, class_name):
    annotations = []
    
    for obj in data.findall(class_name):
        xmin = float(obj.attrib['xmin'])
        ymin = float(obj.attrib['ymin'])
        xmax = float(obj.attrib['xmax'])
        ymax = float(obj.attrib['ymax'])

        x_center = (xmin + xmax) / 2 / img_width
        y_center = (ymin + ymax) / 2 / img_height
        box_width = (xmax - xmin) / img_width
        box_height = (ymax - ymin) / img_height

        row = f"{class_id} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}"
        annotations.append(row)

    return annotations

In [ ]:
nothing = []

for page in pages:
    img_width = float(page.attrib['width'])
    img_height = float(page.attrib['height'])
    page_number = page.attrib['index']

    yolo_annotations = []

    text = annotation(page, img_width, img_height, 0, 'text')
    frame = annotation(page, img_width, img_height, 1, 'frame')

    yolo_annotations += text
    yolo_annotations += frame

    if (len(yolo_annotations) == 0):
        nothing.append(page_number)
    else:
        save_annotation('YasasiiAkuma', page_number, yolo_annotations)   


In [72]:
nothing

['0', '1', '24', '45', '93']

In [ ]:
remove_no_labels(manga_name='YasasiiAkuma', no_label=nothing)

In [ ]:
BASE_PATH_ANNOTATION = r"D:\Project\Manga109\Manga109_released_2023_12_07\images"
os.listdir(BASE_PATH_ANNOTATION)

In [88]:
import uuid

uid = uuid.uuid4()

uid

UUID('4167a862-653e-4d28-98a3-054d98eec269')

In [94]:
manga_list = os.listdir(f'{BASE_PATH}\\images_all')
label_list = os.listdir(f'{BASE_PATH}\\labels_all')

In [96]:
manga_list == label_list

True

In [105]:
BASE = r"D:\Project\Manga109\dataset"

train = os.listdir(f'{BASE}\\test\\images')
train1 = os.listdir(f'{BASE}\\test\\labels')

In [106]:
len(train1)

1014